In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('event_level_data_dirty.csv')
df.head()

,record_id,timestamp,day_of_week,hour_of_day,is_weekend,is_public_holiday,lat,long,weather,temperature,humidity,location_id,location_name,timezone_info
0,1,2025-01-01 07:01:00,2.0,7.0,False,True,1.280884,{}103.869885{},rainy,24.8,93.5,0,SEMBAWANG EATING HOUSE,NaN
1,2,2025-01-01 07:16:00,2.0,7.0,\r\nFalse\r\n,\nTrue\n,1.280884,103.869885,rainy,60.0,91.6,0,SEMBAWANG EATING HOUSE,NaN
2,3,NaN,2.0,7.0,False,NaN,1.280884,103.869885,cloudy,()23.7,86.9,0,SEMBAWANG EATING HOUSE,NaN
3,4,2025-01-01 07:38:00,2.0,7.0,@False,True,1.280884,103.869885,cloudy,24.2,85.7,0,SEMBAWANG EATING HOUSE,NaN
4,5,2025-01-01 07:39:00,NaN,7.0,\r\nFalse\r\n,True,\r\n1.280884\r\n,103.869885,rainy,24.7,91().2,0,SEMBAWANG EATING HOUSE,NaN


In [42]:
def clean_data(input_path, output_path):
    # Load data
    df = pd.read_csv(input_path)

    # 1. Drop duplicates and unnecessary columns
    # The dirty file has extra rows for the same record_id; keep the first occurrence
    df = df.drop_duplicates(subset=['record_id'], keep='first')
    if 'timezone_info' in df.columns:
        df = df.drop(columns=['timezone_info'])

    # 2. Clean String Noise
    # This removes common symbols like {}, (), @, *, etc. and fixes accented characters
    def basic_clean(text):
        if pd.isna(text): return text
        text = str(text).strip().lower()
        # Remove special characters
        text = re.sub(r'[{}()\[\]@*&|°\\ø$%//#/]', '', text)
        # Fix specific variations (e.g., 'fálsë' -> 'false')
        text = text.replace('á', 'a').replace('ë', 'e').replace('ï', 'i').replace('ü', 'u').replace('û', 'ou')
        return text

    for col in ['weather', 'is_weekend', 'is_public_holiday', 'location_name']:
        df[col] = df[col].apply(basic_clean)

    # 3. Handle Timestamps and Date Features
    # Fill missing timestamps based on sequential record_ids
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['timestamp'] = df['timestamp'].interpolate(method='linear')

    # Recalculate time features to ensure consistency
    df['day_of_week'] = df['timestamp'].dt.dayofweek
    df['hour_of_day'] = df['timestamp'].dt.hour
    df['is_weekend'] = df['day_of_week'].isin([5, 6])

    # 4. Clean Numeric Columns (Lat, Long, Temp, Humidity)
    def clean_numeric(val):
        if pd.isna(val): return np.nan
        # Extract digits, dots, and minus signs only
        clean_val = re.sub(r'[^0-9.\-]', '', str(val))
        try:
            return float(clean_val)
        except:
            return np.nan

    num_cols = ['lat', 'long', 'temperature', 'humidity']
    for col in num_cols:
        df[col] = df[col].apply(clean_numeric)

    # 5. Impute Location Data using Location ID
    # Use location_id to fix incorrect Lat/Long/Name (e.g., -100.0 or 999.0)
    for col in ['lat', 'long', 'location_name']:
        # Create a mapping of location_id to the most common (mode) valid value
        mapping = df[df[col].notna() & (df[col] != 0) & (df[col] != -100) & (df[col] != 999)] \
                    .groupby('location_id')[col].agg(lambda x: x.value_counts().index[0])
        df[col] = df['location_id'].map(mapping)

    # 6. Handle Outliers in Environment Data
    # Cap temperatures to reasonable ranges (e.g., 20-40) or interpolate
    df.loc[(df['temperature'] < 15) | (df['temperature'] > 45), 'temperature'] = np.nan
    df['temperature'] = df.groupby('location_id')['temperature'].transform(lambda x: x.interpolate().ffill().bfill())
    df['humidity'] = df.groupby('location_id')['humidity'].transform(lambda x: x.interpolate().ffill().bfill())

    # 7. Final Formatting
    df['is_public_holiday'] = df['is_public_holiday'].map({'true': True, 'false': False}).fillna(False).astype(bool)
    df['location_id'] = df['location_id'].astype(int)

    # Sort and Save
    df = df.sort_values('record_id')
    df.to_csv(output_path, index=False)
    return df

In [43]:
# Run the cleaning
clean_df_result = clean_data('event_level_data_dirty.csv', 'restored_clean_data.csv')

error: unterminated character set at position 0

In [36]:
clean_df_result.head()

,record_id,timestamp,day_of_week,hour_of_day,is_weekend,is_public_holiday,lat,long,weather,temperature,humidity,location_id,location_name
0,1,2025-01-01 07:01:00,2,7,0,1,1.280884,103.869885,rainy,24.80,93.5,0,sembawang eating house
1,2,2025-01-01 07:16:00,2,7,0,1,1.280884,103.869885,rainy,24.25,91.6,0,sembawang eating house
2,3,2025-01-01 07:27:00,2,7,0,0,1.280884,103.869885,cloudy,23.70,86.9,0,sembawang eating house
3,4,2025-01-01 07:38:00,2,7,0,1,1.280884,103.869885,cloudy,24.20,85.7,0,sembawang eating house
4,5,2025-01-01 07:39:00,2,7,0,1,1.280884,103.869885,rainy,24.70,91.2,0,sembawang eating house


In [28]:
clean_df_result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 211807 entries, 0 to 211806
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   record_id          211807 non-null  int64         
 1   timestamp          211807 non-null  datetime64[ns]
 2   day_of_week        211807 non-null  int32         
 3   hour_of_day        211807 non-null  int32         
 4   is_weekend         211807 non-null  int64         
 5   is_public_holiday  211807 non-null  int64         
 6   lat                211807 non-null  float64       
 7   long               211807 non-null  float64       
 8   weather            201432 non-null  object        
 9   temperature        211807 non-null  float64       
 10  humidity           211807 non-null  float64       
 11  location_id        211807 non-null  int64         
 12  location_name      211807 non-null  object        
dtypes: datetime64[ns](1), float64(4), int32(2), int64

In [35]:
# Changing True/False values to numeric 1/0
clean_df_result['is_weekend'] = clean_df_result['is_weekend'].astype(int)
clean_df_result['is_public_holiday'] = clean_df_result['is_public_holiday'].astype(int)

In [41]:
clean_df_result['weather'].unique()

array(['rainy', 'cloudy', nan, 'rain', 'cludy', ' \trainy', 'clear',
       'night_clear', 'rainy  ', '\nrainy\n', 'cloudy \t', 'rainy\t ',
       '\nrainy', 'rainy \t', ' \trainy \t', '\trainy\t', '  rainy  ',
       '  cloudy', 'rainy\t', 'cloudy\n', '\tcloudy', '\ncloudy',
       'night clear', '\t clear', '\r\ncloudy', 'clear\r\n', '\ncloudy\n',
       '\r\ncloudy\r\n', 'cloudy  ', '\tnight_clear', 'cloudy\r\n',
       '  clear', '\r\nrainy\r\n', ' \tcloudy \t', 'cloudy\t',
       '  clear  ', ' \tcloudy', '\t cloudy\t ', '  cloudy  ', 'clear\n',
       '\tcloudy\t', '\t cloudy', '\t night_clear', '  rainy', '\t rainy',
       'cloudy\t ', '\trainy', 'rainy\n', 'rain\t', '\t rainy\t ',
       'cludy\t ', 'clear\t ', '\r\nrainy', ' \tclear \t',
       'night_clear\t', 'rainy\r\n', '  rain', 'clear\t', ' \train',
       '\tclear', 'night_clear\t ', '\nnight_clear', '\r\nclear\r\n',
       '\nrain\n', 'night_clear\r\n', '\tnight_clear\t', '\nclear',
       'clear  ', '\t night clear\t

In [ ]:
# Changing weather values into numeric values
